In [ ]:
class ERPCov():
    def __init__(self):
        self.R = None

    def CovMatrix(self, ts):
        R = 1/(ts.shape[1]-1) * ts @ ts.T
        return R
        

    def get_ts0(self, X: list, y: np.array):
        ts0 = 0
        k = 0
        for ts, label in zip(X, y):
            ts0 += (1-label) * ts
            k += 1-label
        return ts0 / k
    

    def get_ts1(self, X: list, y: np.array):
        ts1 = 0
        k = 0
        for ts, label in zip(X, y):
            ts1 += label * ts
            k += label
        return ts1 / k
    

    # y подается только на обучении
    def get_CovMatrices(self, X: list, y=None):
        # создаем переменные средние по классам, которые будет исползовать на тесте
            
        Rs = []
        for ts in X:
            R = self.CovMatrix(ts)
            Rs.append(R)
        if y is not None:
            self.Rs = Rs
        else:
            return Rs
        

    # Алгоритм 1. 
    
    def Exp(self, R, pi):
        eigenvalues, eigenvectors = np.linalg.eigh(R)
        sqrt_eigenvalues = np.diag(np.sqrt(eigenvalues))
        sqrt_R = eigenvectors @ sqrt_eigenvalues @ np.linalg.inv(eigenvectors)

        Q = np.linalg.inv(sqrt_R) @ pi @ np.linalg.inv(sqrt_R)
        eigenvalues, eigenvectors = np.linalg.eigh(Q)
        exp_eigenvalues = np.diag(np.exp(eigenvalues))
        exp_Q = eigenvectors @ exp_eigenvalues @ np.linalg.inv(eigenvectors)

        return sqrt_R @ exp_Q @ sqrt_R
    

    def Log(self, R, R_k):
        eigenvalues, eigenvectors = np.linalg.eigh(R)
        sqrt_eigenvalues = np.diag(np.sqrt(eigenvalues))
        sqrt_R = eigenvectors @ sqrt_eigenvalues @ np.linalg.inv(eigenvectors)

        Q = np.linalg.inv(sqrt_R) @ R_k @ np.linalg.inv(sqrt_R)
        eigenvalues, eigenvectors = np.linalg.eigh(Q)
        log_eigenvalues = np.diag(np.log(eigenvalues))
        log_Q = eigenvectors @ log_eigenvalues @ np.linalg.inv(eigenvectors)

        return sqrt_R @ log_Q @ sqrt_R


    def Mean_of_SPD_Matrices(self, Rs=None, n_iter=50, eps=1e-11):
        if Rs is None:
            Rs = self.Rs
        R = 1/len(Rs) * sum(Rs)
        
        for _ in tqdm(range(n_iter)):
            list_pi = [self.Log(R, Rs[k]) for k in range(len(Rs))]
            pi = 1/len(list_pi) * sum(list_pi)
            R = self.Exp(R, pi)
            if np.linalg.norm(pi, 'fro') < eps:
                break
        self.R = R


    def Tangent_projection(self, X=None):
        if X is None:
            Rs = self.Rs
        else:
            Rs = self.get_CovMatrices(X)

        pis = []
        for R_k in Rs:
            pi = self.Log(self.R, R_k)
            pis.append(pi)
        return pis

    
    def fit(self, X: list, y: np.array):
        self.get_CovMatrices(X, y)
        self.Mean_of_SPD_Matrices()



    def transform(self, X: list):
        if self.R is None:
            raise Exception('First you need to train the model with method fit')
        
        pis = self.Tangent_projection(X)
        output = []
        for pi in tqdm(pis):
            output_vector = []
            for i in range(pi.shape[0]):
                for j in range(pi.shape[1]):
                    if i <= j:
                        if i == j:
                            output_vector.append(pi[i, j])
                        else:
                            output_vector.append(np.sqrt(2) * pi[i, j])
            output.append(np.array(output_vector))
        return np.array(output)
            

    def fit_transform(self, X: list, y: np.array):
        self.fit(X, y)
        self.transform(X)

In [ ]:
preprocess = ERPCov()

preprocess.get_CovMatrices(ts_train, y_train)

preprocess.fit(ts_train, y_train)
X_train = preprocess.transform(ts_train)